In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import network_generation as homomul
import spread_simulation
import homophily_multi_attr_viz as homomul_viz

In [3]:
model_settings = {
    'N': 3200, 
    'm': 5, 
    'consolidation_param': 0.1,
    'marginal_distribution': [[0.5,0.5],[0.5,0.5]], 
    'homophily': 0.8, 
    'complex_threshold': 0.16
}

experiment_settings = {
    'n_networks': 3,
    'n_initial_seeds': 10
}

def perform_experiment(N: int, m: int, consolidation_param: float, homophily: float,
                       marginal_distribution: list = [[0.5,0.5],[0.5,0.5]], complex_threshold: float = 0.16, 
                       n_networks: int = 3, n_initial_seeds: int = 10) -> list[float]: 
    
    comp_pop_frac_tnsr = homomul.consol_comp_pop_frac_tnsr(marginal_distribution, consolidation_param)

    h1 = np.array([[homophily, 1-homophily], [1-homophily, homophily]])
    h2 = h1.copy()
    h_mtrx_lst = np.array([h1,h2])

    results = []

    for _ in range(n_networks): 
        g = homomul.am_v2(
                    h_mtrx_lst,
                    comp_pop_frac_tnsr,
                    homophily_kind='all',
                    directed=False,
                    pop_fracs_lst = marginal_distribution,
                    N=N,
                    m=m
                    )
        
        initial_seeds = np.random.choice(g.nodes, n_initial_seeds, replace=False)
        for initial in initial_seeds: 
        
            initial = [initial] + [x for x in g.neighbors(initial)]
            g = spread_simulation.complex_spread(g, initial, 0.16)
            results.append(spread_simulation.fraction_infected(g))

    return np.average(results)

perform_experiment(**model_settings, **experiment_settings)

0.14252083333333332

In [ ]:
homophily_settings = np.linspace(0,1,11)
consolidation_settings = np.linspace(0,0.5, 6)

results = np.zeros((homophily_settings.size(), consolidation_settings.size()))
for i, homophily in enumerate(homophily_settings): 
    for j, consolidation_param in enumerate(consolidation_settings):

        print(f'homophily: {homophily} / consolidation: {consolidation_param}')
        model_settings['homophily'] = homophily
        model_settings['consolidation_param'] = consolidation_param

        r = perform_experiment(**model_settings, **experiment_settings)
        results[i, j] = r
